# Mount and module/library setup

In this step, the shortcut setup in your Google Drive will be mounted to this Google Colab Notebook in order for access. It is important the steps were followed correctly in the user guide and file names are not changed.

There will be a pop-up window in which Google Colab will request access to your Google Drive, this is normal and must be accepted to progress.

In [ ]:
#Import google collab drive usage, mount then enter directory for file access.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/adamcao-1906735-project

!pip install -q tensorflow-ranking tensorflow-serving-api
!pip install -U tensorflow_text
!pip install -q tf-models-official==2.4.0
!pip install -Uq grpcio==1.26.0

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

import pathlib
import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_text as tf_text
from tensorflow_serving.apis import input_pb2
from google.protobuf import text_format
import bz2
import json
import pandas as pd
import re
import os
from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

# Training Evaluation Using TensorBoard

Once model training is complete, the training process can be evaluated by using Tensorboard and loading in the training data generated. This can be done by running the following script:

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="/content/drive/MyDrive/adamcao-1906735-project/models/SampleModel/"
#%tensorboard --logdir="/content/drive/MyDrive/adamcao-1906735-project/models/NDCGTitle31256Final/"
#%tensorboard --logdir="/content/drive/MyDrive/adamcao-1906735-project/models/NDCGTitleTags31256Final/"

# Start Tensorflow Serving Server

This starts the Tensorflow Serving server using the model you wish use in order to make predictions, it will choose the best model by loss in the training iterations.

In [ ]:
%%bash --bg
%cd /
tensorflow_model_server \
  --port=8500 \
  --rest_api_port=8501 \
  --model_name=tfrbert \
  --model_base_path=/content/drive/MyDrive/adamcao-1906735-project/models/SampleModel/export/best_model_by_loss >server.log 2>&1

# Make and export predictions

As the Tensorflow Serving Server is now running, this calls the prediction script using the Test data, formatted previously using the model chosen when starting the serving server. Sequence length should match the sequence length the model was trained on.

It will export the predictions in the output folder inside the folder of the model.

A successful example output:

```
 * Running with arguments: Namespace(do_lower_case=False, input_file='FormattedData/testJSON.json', output_file='SampleModel/output/testscores.json', sequence_length=256, vocab_file='cased_L-12_H-768_A-12/vocab.txt')
 * Generating predictions for JSON ranking problems (filename: FormattedData/MainTest.json)

Predicting 96 / 96 (100.00%)
 * exportRankingOutput(): Exporting scores to JSON (SampleModel/output/testscores.json)
 * Total execution time: 1:43:01.389
```



In [ ]:
outputFolder = 'models/SampleModel/output/'
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

!BERT_DIR="cased_L-12_H-768_A-12"  && \
sudo python bertPython/tfrbert_client_predict_from_json.py \
    --vocab_file=${BERT_DIR}/vocab.txt \
    --sequence_length=256 \
    --input_file=FormattedData/MainTest.json \
    --output_file=models/SampleModel/output/testscores.json

# nDCG Scoring Evaluation of Models

Using the functions below, it runs a nDCG scoring evaluation of each model in the /models folder and prints the nDCG scores for each one. It uses the predicted results from the exported testscores.json produced from the Serving server.

In [ ]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np

#https://finisky.github.io/2019/04/24/ndcg/
import math,os

def NdcgFunc(rankedDataset, p, customGain = None):
    if len(rankedDataset) == 0:
        return 0

    gain = customGain
    if gain == None:
        gain = [(pow(2, i) - 1) for i in range(p)]

    ndcgSum = 0
    count = 0
    for singleList in rankedDataset:
        dcg = 0
        for i in range(min(len(singleList), p)):
            dcg += gain[singleList[i]] / math.log(i + 2)

        idcg = 0
        optimalOrder = sorted(singleList, reverse = True)
        for i in range(min(len(optimalOrder), p)):
            if i >= p:
                break
            idcg += gain[optimalOrder[i]] / math.log(i + 2)

        if abs(idcg) < 0.0001:
            continue

        count += 1
        ndcgSum += dcg / idcg

    return ndcgSum / count

ndcg10Dict = {}

def ndcgCalc(k, file):
  predictDataDir = "models/" + file + "/output/"
  g = open(predictDataDir + 'testscores.json')
  testDataPred = json.load(g)
  g.close()

  predRelList = []
  for x in testDataPred['rankingProblemsOutput']:
    relList = []
    for z in x['documents']:
      relList.append(z['relevance'])
    predRelList.append(relList)

  ndcgs = []
  ndcg10s = []
  for x in predRelList:
    # Relevance scores in output order
    relevance_scores = [x]

    if len(x) > 1 and (1 in x or 2 in x):
      if k == -1:
        ndcgs.append( NdcgFunc(relevance_scores, len(x) ))
      else:
        ndcgs.append( NdcgFunc(relevance_scores,k) )
        if k == 10:
          ndcg10s.append(NdcgFunc(relevance_scores,k))
    else:
      ndcg10s.append(0.0)

    ndcg10Dict[file] = ndcg10s
  return "Avg - " + str(round(np.average(ndcgs), 3 )) #+ " Max:" + str(np.max(ndcgs)) + " Min:" + str(np.min(ndcgs))

#For all items use -1
def testeval(file):
  print("========================")
  print(file)
  print("NDCGALL: " + str(ndcgCalc(-1, file)))
  print("NDCG@03: " + str(ndcgCalc(3, file)))
  print("NDCG@05: " + str(ndcgCalc(5, file)))
  print("NDCG@10: " + str(ndcgCalc(10, file)))

root='models/'
dirlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]

for x in dirlist:
  testeval(x)

# nDCG@10 Histogram evaluation

Here we can plot the histogram of the results produced by a specific model, showing all ranking problems and how it scored in each one.

In [ ]:
# Import the libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Number of topics in data
print(len(ndcg10Dict['SampleModel']))
# matplotlib histogram
plt.hist(np.asarray(ndcg10Dict['SampleModel'], dtype='float'), color = 'orange', edgecolor = 'black', bins = 96)

# Add labels
plt.title('Histogram of nDCG@10 scores')
plt.xlabel('nDCG@10')
plt.ylabel('Number of queries')